### Install Packages

In [6]:
# Install packages
%pip install openai-whisper yt-dlp sentence-transformers pandas -q
%pip install youtube-transcript-api --upgrade
%pip install ffmpeg-python -q


[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached youtube_transcript_api-1.2.4-py3-none-any.whl.metadata (24 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
Using cached youtube_transcript_api-1.2.4-py3-none-any.whl (485 kB)
Using cached defusedxml-0.7.1-py2.py3-none-any.whl (25 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [youtube-transcript-api]

[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to

In [7]:
%pip install requests -q
%pip install transformers -q
%pip install torch -q

%pip install scikit-learn -q


[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
import re
import json
import requests
from urllib.parse import urlparse
from typing import Dict, List
from IPython.display import display, Markdown

import pandas as pd
import whisper
import yt_dlp
from sentence_transformers import SentenceTransformer

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Model

In [9]:
# Load Whisper model for audio transcription
whisper_model = whisper.load_model("base")

# Load SentenceTransformer for embedding recipes
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Recipe store to hold ingested recipes (mock DB)
recipe_store = []

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 372.90it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


### Youtube Scrapping - v2

References: https://colab.research.google.com/drive/1KOlGHd3Rqs_w6_6F_LiqTF1Dcga_XZBk?usp=sharing#scrollTo=jBTs4wnBnH1G

In [10]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound
from urllib.parse import urlparse, parse_qs
import glob

def get_video_id(link: str) -> str:
    parsed = urlparse(link)
    if 'youtube.com' in parsed.netloc:
        return parse_qs(parsed.query).get('v', [''])[0]
    elif 'youtu.be' in parsed.netloc:
        return parsed.path.lstrip('/')
    return ""

def try_youtube_captions(video_id: str) -> str:
    try:
        transcript = YouTubeTranscriptApi().get_transcript(video_id)
        return " ".join([t["text"] for t in transcript])
    except TranscriptsDisabled:
        print("⚠️ Captions are disabled for this video.")
    except NoTranscriptFound:
        print("⚠️ No transcript found.")
    except Exception as e:
        print(f"❌ Caption extraction failed: {e}")
    return ""

def transcribe_audio_with_yt_dlp(link: str, use_cookies: bool = False, cookies_path: str = "youtube_cookies.txt") -> str:
    ydl_opts = {
        'format': 'bestaudio/best',
        'quiet': False,
        'outtmpl': '/tmp/audio.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }
    if use_cookies:
        ydl_opts['cookiefile'] = cookies_path

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([link])

        audio_files = glob.glob("/tmp/audio*.mp3")
        if not audio_files:
            raise FileNotFoundError("No audio file found.")
        audio_path = audio_files[0]

        print(f"🎧 Transcribing audio: {audio_path}")
        result = whisper_model.transcribe(audio_path)
        return result['text']
    except Exception as e:
        print(f"❌ Audio transcription failed: {e}")
        return ""

def ingest_recipe_smart(link: str, cookies_path: str = "youtube_cookies.txt") -> str:
    video_id = get_video_id(link)
    if not video_id:
        return "Invalid YouTube link."

    print("🔍 Trying captions...")
    text = try_youtube_captions(video_id)
    if text:
        print("✅ Captions found.")
        return text

    print("🌀 Trying audio transcription without cookies...")
    text = transcribe_audio_with_yt_dlp(link, use_cookies=False)
    if text:
        print("✅ Transcription without cookies succeeded.")
        return text

    print("🔐 Trying with cookies...")
    text = transcribe_audio_with_yt_dlp(link, use_cookies=True, cookies_path=cookies_path)
    if text:
        print("✅ Transcription with cookies succeeded.")
        return text

    return "❌ All attempts failed."

In [11]:
# TEST-v2
# link = "https://youtu.be/2uYoqclu6so"
link = "https://youtu.be/2uYoqclu6so?si=4au4jhBN33YT60OQ"
text = ingest_recipe_smart(link)
print(text[:500])

🔍 Trying captions...
❌ Caption extraction failed: 'YouTubeTranscriptApi' object has no attribute 'get_transcript'
🌀 Trying audio transcription without cookies...
[youtube] Extracting URL: https://youtu.be/2uYoqclu6so?si=4au4jhBN33YT60OQ
[youtube] 2uYoqclu6so: Downloading webpage


[youtube] 2uYoqclu6so: Downloading android vr player API JSON
[info] 2uYoqclu6so: Downloading 1 format(s): 251
[download] Destination: /tmp/audio.webm
[download] 100% of    6.44MiB in 00:00:00 at 16.50MiB/s  
[ExtractAudio] Destination: /tmp/audio.mp3
Deleting original file /tmp/audio.webm (pass -k to keep)
🎧 Transcribing audio: /tmp/audio.mp3


/opt/homebrew/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ Transcription without cookies succeeded.
 Hey guys, today I'm going to show you how to make the easiest baked salmon recipe. This recipe really is foolproof. It takes less than 20 minutes to make and it gives you an incredibly moist, flaky and flavorful salmon. This is hands down the salmon recipe that I make most often in my house and it just makes for an easy, healthy dinner recipe that you can serve with a variety of sides. So let me show you how to make it. On Instagram I asked you if you wanted to see the salmon recipe with indivi


In [12]:
import sys
print(sys.executable)

/opt/homebrew/opt/python@3.11/bin/python3.11


#### YouTube Scrapping - v1

In [ ]:
import glob
import os
import shutil

def is_youtube_link(link: str) -> bool:
    parsed = urlparse(link)
    return "youtube.com" in parsed.netloc or "youtu.be" in parsed.netloc

def transcribe_youtube(link: str, cookies_path: str = "youtube_cookies.txt") -> str:
    ydl_opts = {
        'format': 'bestaudio/best',
        'quiet': False,
        'outtmpl': '/tmp/audio.%(ext)s',  # Allow yt-dlp to set proper extension
        'cookiefile': cookies_path,
        'postprocessors': [{  # Extract audio as mp3
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([link])

        # Find the downloaded mp3 file in /tmp
        files = glob.glob("/tmp/audio*.mp3")
        if not files:
            raise FileNotFoundError("Audio file not found after download")
        audio_path = files[0]

        print(f"Using audio file: {audio_path}")

        result = whisper_model.transcribe(audio_path)
        return result['text']

    except Exception as e:
        print(f"❌ Error downloading/transcribing: {str(e)}")
        return ""



def extract_recipe_from_text(text: str) -> Dict:
    steps = re.split(r"\d+\.", text)
    steps = [step.strip() for step in steps if len(step.strip()) > 10]
    return {
        "raw_text": text,
        "steps": steps,
        "num_steps": len(steps),
        "tags": [],  # Placeholder for later
    }


def store_recipe(recipe_data: Dict):
    embedding = embedding_model.encode(recipe_data['raw_text'])
    recipe_data['embedding'] = embedding
    recipe_store.append(recipe_data)


In [ ]:
def ingest_recipe_from_youtube(link: str, cookies_path: str = "youtube_cookies.txt"):
    if not is_youtube_link(link):
        return "Invalid YouTube URL."

    text = transcribe_youtube(link, cookies_path)
    if not text:
        return "Failed to transcribe video."

    display(Markdown(f"### Transcribed Text\n{text[:500]}..."))

    recipe = extract_recipe_from_text(text)
    store_recipe(recipe) # Use store_recipe here

    display(Markdown("### Extracted Steps"))
    for i, step in enumerate(recipe['steps']):
        display(Markdown(f"**Step {i+1}:** {step}"))

    return recipe

In [ ]:
# Sample test link
youtube_link = "https://youtu.be/2uYoqclu6so?si=4au4jhBN33YT60OQ"
result = ingest_recipe_from_youtube(youtube_link)

[youtube] Extracting URL: https://youtu.be/2uYoqclu6so?si=4au4jhBN33YT60OQ
[youtube] 2uYoqclu6so: Downloading webpage


[youtube] 2uYoqclu6so: Downloading android sdkless player API JSON
[youtube] 2uYoqclu6so: Downloading web safari player API JSON


[youtube] 2uYoqclu6so: Downloading m3u8 information


[info] 2uYoqclu6so: Downloading 1 format(s): 251
[download] Destination: /tmp/audio.webm
[download] 100% of    6.44MiB in 00:00:00 at 23.82MiB/s  
[ExtractAudio] Destination: /tmp/audio.mp3
Deleting original file /tmp/audio.webm (pass -k to keep)
Using audio file: /tmp/audio.mp3


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


### Transcribed Text
 Hey guys, today I'm going to show you how to make the easiest baked salmon recipe. This recipe really is foolproof. It takes less than 20 minutes to make and it gives you an incredibly moist, flaky and flavorful salmon. This is hands down the salmon recipe that I make most often in my house and it just makes for an easy, healthy dinner recipe that you can serve with a variety of sides. So let me show you how to make it. On Instagram I asked you if you wanted to see the salmon recipe with indivi...

### Extracted Steps

**Step 1:** Hey guys, today I'm going to show you how to make the easiest baked salmon recipe. This recipe really is foolproof. It takes less than 20 minutes to make and it gives you an incredibly moist, flaky and flavorful salmon. This is hands down the salmon recipe that I make most often in my house and it just makes for an easy, healthy dinner recipe that you can serve with a variety of sides. So let me show you how to make it. On Instagram I asked you if you wanted to see the salmon recipe with individual fillets or one large half a salmon and individual fillets one. So that's what I'm using today. But just know that you can use the same recipe on one large piece of salmon as well. Take the salmon out of the fridge and let it come to room temperature for about 15 minutes before cooking. For the same reason that we do this for chicken or steak, it just helps to ensure that the salmon will cook evenly. I always cook salmon with the skin on as it's much easier to remove it after the salmon has cooked and today I'm using these gorgeous and thick King Salmon Follets. I talk about the different varieties of salmon on a separate blog post on my website and I'll link to that below. But King Salmon is known for being thick, super moist and having a buttery smooth texture and that's because it has a higher fat content. It's the most expensive variety of salmon and definitely a splurge but it's so good. Of course, this recipe works just as great with cojo and sake salmon. So just grab what's in season, what's in your budget or what you have access to. And lastly for the individual fillets, try to keep them about the same size so again, they're more apt to cook evenly in the oven. Preheat your oven to 375 degrees Fahrenheit and then get a baking dish or rim to baking sheet out. I like to lay down parchment paper so that the salmon removes easily but to be honest, because we're cooking the salmon at a gentle temperature, it rarely sticks either way. Some recipes say to cook salmon at a high temperature like 425 degrees Fahrenheit, but I prefer 375 for extremely moist and succulent salmon that's less likely to overcook and dry out. So place the flays in your baking dish and once that's done, set them aside. Melt two tablespoons of butter and I just do this in the microwave for about 20 seconds. You want it melted but not hot and certainly not spattering or exploding in your microwave. So 10 to 20 seconds usually does the trick. And then to that you'll add half a tablespoon or so of lemon juice. I don't usually measure this, I just squirt the juice from a quarter slice of a lemon straight into the bowl and catch any seeds in my hands. The measurement usually ends up close enough. Stir the lemon butter mixture together and then brush it generously all over the salmon. And if you don't have a brush, you could just spoon it over the top as well. Next, mince or finely chopped 3 garlic cloves and then sprinkle them all over the top of the salmon. I suppose you could also stir this in with the butter mixture if you don't want garlicy hands but I've always just sprinkled it on top so that it's evenly spread out. And if you are worried about garlicy hands, there's a few really simple tricks to removing that garlic smell. Like rubbing salt and baking soda between your hands, rubbing coffee grounds between them, or rubbing your hands on something stainless steel like a pot or utensil. And rubbing your hands on a stainless steel sink works if you happen to have one. But once your garlic is nicely dappled all over the salmon, sprinkle the top with salt and cracked black pepper. You can lay those extra lemon slices all around the salmon. I don't have a lot of extra room today as my salmon pieces are so big, but once these lemon slices have baked, they're soft and juicy and great for squirting on top of the salmon. Make the salmon for 12 to 15 minutes depending on the thickness of your salmon. And always, always set a timer so that you don't accidentally forget about it because you don't want it overcooked. While the salmon is baking, I'll quickly chop up some fresh herbs. Now if you're wondering why I didn't add them to the salmon before going in the oven, it's because fresh herbs can burn quickly. Parsley is a little bit more durable, but dill will singe pretty fast and then you'll have burned dill bits on top and that's not very tasty. So it's best to just sprinkle the fresh herbs on top of the salmon when it comes out of the oven for the freshest flavor, and I'm chopping up about a teaspoon each of parsley and dill. When the salmon comes out of the oven, it should look juicy and moist and the color will be a soft, orangey pink. You might also notice some white bits coming out of the salmon. Many people think that this is fat, but it's not. It's called albumin and it's actually protein that sort of pushes out of the salmon as it's heated up in cooks. Different varieties of salmon and different cooking methods will affect how much albumin you see, but a more gentle oven temperature like what we're using today helps to reduce it and keep the salmon more moist. Long story short, it's totally normal and safe, but you can always scrape it off the sides before serving as well. Now is the time to put the finishing touch on the baked salmon and sprinkle those fresh herbs. The simple combination of garlic, butter, lemon, and herbs is a classic and it pairs so well with a variety of different sides. Today I'm serving it up with creamy mashed potatoes and green beans and both of these recipes are on my website as well. You'll know the salmon is perfectly cooked when it flakes easily with a fork and it's still moist on the inside. King salmon is great because even if you slightly overcook it, it stays so darn juicy and tender. It really is full proof. Oh, and don't forget to squirt a little bit of lemon juice on top, which I almost forgot, and then dig into this healthy dinner. I hope you enjoyed this recipe and if you did, make sure to give it a thumbs up, share it with your family and friends, and I will see you again in the next video.

### Instagram Scrapping

In [ ]:
!pip install instaloader

In [ ]:
import instaloader

# Create instaloader instance
L = instaloader.Instaloader(download_video_thumbnails=False, download_geotags=False, download_comments=False)


In [ ]:
# Only needed if you want to access private content or saved reels
# L.login('your_username', 'your_password')  # Optional

In [ ]:
def extract_instagram_text(link: str) -> str:
    shortcode = None
    match = re.search(r"instagram\.com/(reel|p)/([a-zA-Z0-9_-]+)", link)
    if match:
        shortcode = match.group(2)
    else:
        return "Invalid Instagram link."

    post = instaloader.Post.from_shortcode(L.context, shortcode)

    caption = post.caption or "No caption available."
    video_url = post.video_url if post.is_video else None

    # Optional: download video audio for Whisper transcription
    if video_url:
        audio_path = "/tmp/insta_audio.mp3"
        os.system(f"yt-dlp -x --audio-format mp3 -o '{audio_path}' '{video_url}'")
        transcript = whisper_model.transcribe(audio_path)['text']
        return f"{caption}\n\nTranscribed Audio:\n{transcript}"
    else:
        return caption

In [ ]:
def ingest_recipe_from_instagram(link: str):
    print("Fetching Instagram content...")
    text = extract_instagram_text(link)
    display(Markdown(f"### Extracted/Transcribed Text\n{text[:500]}..."))

    recipe = extract_recipe_from_text(text)
    store_recipe(recipe)

    display(Markdown("### Extracted Steps"))
    for i, step in enumerate(recipe['steps']):
        display(Markdown(f"**Step {i+1}:** {step}"))

    return recipe

In [ ]:
# Example Reel/Post
insta_link = "https://www.instagram.com/reel/DGtu1YbPwd9/?igsh=MTBhcDk1Ymk2ZzFoYg=="
result = ingest_recipe_from_instagram(insta_link)

Fetching Instagram content...


JSON Query to graphql/query: Expecting value: line 1 column 1 (char 0) [retrying; skip with ^C]
JSON Query to graphql/query: 401 Unauthorized - "fail" status, message "Please wait a few minutes before you try again." when accessing https://www.instagram.com/graphql/query?variables=%7B%22shortcode%22%3A%22DGtu1YbPwd9%22%7D&doc_id=8845758582119845&server_timestamps=true [retrying; skip with ^C]


ConnectionException: JSON Query to graphql/query: 401 Unauthorized - "fail" status, message "Please wait a few minutes before you try again." when accessing https://www.instagram.com/graphql/query?variables=%7B%22shortcode%22%3A%22DGtu1YbPwd9%22%7D&doc_id=8845758582119845&server_timestamps=true

### Text Scrapping

In [ ]:
def ingest_recipe_from_text(raw_text: str):
    display(Markdown("### Input Recipe Text"))
    display(Markdown(raw_text[:500] + ("..." if len(raw_text) > 500 else "")))

    recipe = extract_recipe_from_text(raw_text)
    recipe['raw_text'] = raw_text  # Ensure raw_text is included
    recipe['embedding'] = embedding_model.encode(raw_text)

    # store_recipe_to_vector_db(recipe)

    display(Markdown("### Extracted Steps"))
    for i, step in enumerate(recipe['steps']):
        display(Markdown(f"**Step {i+1}:** {step}"))

    return recipe

In [ ]:
# Paste or type any recipe here
sample_text = """
Masala French Toast

1. Break 2 eggs into a bowl.
2. Add 1 chopped green chili, 1/2 tsp red chili powder, salt, and some chopped cilantro.
3. Mix well with a fork.
4. Dip bread slices into the mixture.
5. Toast both sides on a buttered pan until golden brown.
6. Serve hot with ketchup or chutney.
"""

recipe = ingest_recipe_from_text(sample_text)


### Input Recipe Text


Masala French Toast

1. Break 2 eggs into a bowl.
2. Add 1 chopped green chili, 1/2 tsp red chili powder, salt, and some chopped cilantro.
3. Mix well with a fork.
4. Dip bread slices into the mixture.
5. Toast both sides on a buttered pan until golden brown.
6. Serve hot with ketchup or chutney.


✅ Recipe stored! Total stored: 1


### Extracted Steps

**Step 1:** Masala French Toast

...

**Step 3:** Add 1 chopped green chili, 1/2 tsp red chili powder, salt, and some chopped cilantro.

**Step 4:** Mix well with a fork.

**Step 5:** Dip bread slices into the mixture.

**Step 6:** Toast both sides on a buttered pan until golden brown.

**Step 7:** Serve hot with ketchup or chutney.

### Store Recipes in FAISS + Metadata Index

This step needs to be before ingestion

After ingesting any recipe from any source, it should store it in FAISS immediately for future reference.

STEPS:

1. Setup the FAISS
2. Store the Ingested Recipies from different sources in FAISS

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 45.1 MB/s eta 0:00:00


In [ ]:
import faiss
import numpy as np

# Define embedding dimension (must match your SentenceTransformer)
embedding_dim = 384  # for 'all-MiniLM-L6-v2'

# FAISS index (Flat L2 search for simplicity)
faiss_index = faiss.IndexFlatL2(embedding_dim)

# Metadata store for structured search (can replace with SQLite later)
metadata_store = []


In [ ]:
def store_recipe_to_vector_db(recipe: Dict):
    embedding = recipe['embedding']
    if isinstance(embedding, list):
        embedding = np.array(embedding).astype('float32')

    # Add to FAISS
    faiss_index.add(np.expand_dims(embedding, axis=0))

    # Add to metadata store
    metadata_store.append({
        "text": recipe['raw_text'],
        "steps": recipe['steps'],
        "num_steps": recipe['num_steps'],
        "tags": recipe.get("tags", []),
        # Add more filters later like cuisine, type, time etc.
    })

    print(f"✅ Recipe stored! Total stored: {len(metadata_store)}")


In [ ]:
store_recipe_to_vector_db(recipe)

In [ ]:
# After ingestion:
# YT_recipe = ingest_recipe_from_youtube("https://youtu.be/2uYoqclu6so?si=4au4jhBN33YT60OQ")
# store_recipe_to_vector_db(YT_recipe)
store_recipe_to_vector_db(result)

✅ Recipe stored! Total stored: 2


In [ ]:
# Test function - This is done by the LLM after it gets the query

def search_similar_recipes(query: str, top_k: int = 3):
    query_embedding = embedding_model.encode([query])
    query_embedding = np.array(query_embedding).astype('float32')

    distances, indices = faiss_index.search(query_embedding, top_k)

    print("🔍 Top Similar Recipes:\n")
    for i, idx in enumerate(indices[0]):
        meta = metadata_store[idx]
        print(f"🔸 Match {i+1}: {meta['text'][:300]}...\n---\n")


In [ ]:
# Test function - This is done by the LLM after it gets the query

# search_similar_recipes("I want to make salmon with lime under 30 minutes", top_k=1)
search_similar_recipes("I want to make eggs under 30 minutes", top_k=1)

🔍 Top Similar Recipes:

🔸 Match 1: 
Masala French Toast

1. Break 2 eggs into a bowl.
2. Add 1 chopped green chili, 1/2 tsp red chili powder, salt, and some chopped cilantro.
3. Mix well with a fork.
4. Dip bread slices into the mixture.
5. Toast both sides on a buttered pan until golden brown.
6. Serve hot with ketchup or chutney.
...
---



### Use RAG using Gemini

In [ ]:
!pip install google-generativeai

In [ ]:
import google.generativeai as genai

from google.colab import userdata
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GEMINI_API_KEY)

In [ ]:
def generate_rag_response_with_gemini(user_query: str, top_k: int = 3) -> str:
    # Embed query and search FAISS
    query_embedding = embedding_model.encode([user_query])
    query_embedding = np.array(query_embedding).astype('float32')
    distances, indices = faiss_index.search(query_embedding, top_k)

    # Build context
    context_snippets = []
    for idx in indices[0]:
        recipe = metadata_store[idx]
        snippet = f"Steps:\n" + "\n".join(recipe['steps'][:6])
        context_snippets.append(snippet)

    context = "\n\n---\n\n".join(context_snippets)

    prompt = f"""You are a helpful recipe assistant.

User request: {user_query}

Here are some recipes:

{context}

Please recommend the best-fitting recipe and explain why it was chosen.
"""

    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(prompt)
    return response.text


In [ ]:
query = "Looking for a light Italian appetizer under 30 minutes"
response = generate_rag_response_with_gemini(query)
print(response)

NotFound: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.